In [36]:
import pathlib
import tensorflow as tf
from tensorflow import keras
from keras import layers
# import numpy

img_height = 224
img_width = 224
input_shape = (img_width, img_height, 3)
batch_size = 8
train_dir = pathlib.Path('D:/Thesis_data/training_img')
test_dir  = pathlib.Path('D:/Thesis_data/testing_img')
model_dir = pathlib.Path('D:/Thesis_data/Kaggle_backup/cacao_CAE/model.h5')

In [37]:
train_ds = keras.utils.image_dataset_from_directory(
  train_dir,
  labels="inferred",
  label_mode="categorical",
    seed=252,
  shuffle=True,
    validation_split=0.2,
    subset="training",
  image_size=(img_height, img_width),
  batch_size=batch_size
)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

Found 13272 files belonging to 14 classes.
Using 10618 files for training.


In [38]:
model = keras.models.load_model(model_dir)
model.summary()
# model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


Model: "model_198"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_47 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 rescaling_287 (Rescaling)   (None, 224, 224, 3)       0         
                                                                 
 conv2d_979 (Conv2D)         (None, 218, 218, 84)      12432     
                                                                 
 leaky_re_lu_831 (LeakyReLU)  (None, 218, 218, 84)     0         
                                                                 
 conv2d_980 (Conv2D)         (None, 206, 206, 42)      596274    
                                                                 
 leaky_re_lu_832 (LeakyReLU)  (None, 206, 206, 42)     0         
                                                                 
 max_pooling2d_692 (MaxPooli  (None, 103, 103, 42)     0 

In [39]:
input_layer = layers.Input(shape=input_shape)
input_layer2 = layers.Rescaling(scale=1./255, offset=0)(input_layer)
# encoder
x = layers.Conv2D(84, kernel_size=(7,7))(input_layer2)
x = layers.LeakyReLU(alpha=0.1)(x)

x = layers.Conv2D(42, kernel_size=(13,13))(x)
x = layers.LeakyReLU(alpha=0.1)(x)
x = layers.MaxPooling2D(pool_size=(2,2), strides=2)(x)

x = layers.Conv2D(42, kernel_size=(9,9))(x)
x = layers.LeakyReLU(alpha=0.1)(x)
x = layers.MaxPooling2D(pool_size=(2,2), strides=2)(x)

x = layers.Conv2D(28, kernel_size=(5,5))(x)
x = layers.LeakyReLU(alpha=0.1)(x)
x = layers.MaxPooling2D(pool_size=(2,2), strides=2)(x)

x = layers.Conv2D(28, kernel_size=(3,3))(x)   # (14,14,14)
x = layers.LeakyReLU(alpha=0.1)(x)
x = layers.MaxPooling2D(pool_size=(2,2), strides=2)(x)

x = layers.Conv2D(14, kernel_size=(3,3))(x)   # (7,7,14)
x = layers.LeakyReLU(alpha=0.1)(x)
#     x = layers.MaxPooling2D(pool_size=(2,2), strides=2)(x)


x = layers.Flatten()(x)
x = layers.Dense(686, activation="relu")(x)
x = layers.Dense(343, activation="relu")(x)
x = layers.Dense(112, activation="relu")(x)
# #latent
latent = layers.Dense(56, activation="softmax")(x)
output_layer = layers.Dense(14, activation="softmax")(latent)

cnn = keras.Model(input_layer, output_layer)

for i in range(22):
    cnn.trainable_weights[i] = model.trainable_weights[i]

# cnn.layers.append(layers.Dense(14, activation="softmax"))
cnn.summary()
cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescaling_5 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 conv2d_30 (Conv2D)          (None, 218, 218, 84)      12432     
                                                                 
 leaky_re_lu_30 (LeakyReLU)  (None, 218, 218, 84)      0         
                                                                 
 conv2d_31 (Conv2D)          (None, 206, 206, 42)      596274    
                                                                 
 leaky_re_lu_31 (LeakyReLU)  (None, 206, 206, 42)      0         
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 103, 103, 42)     0   

In [40]:
cnn.save(pathlib.Path('D:/Thesis_data/Kaggle_backup/cacao_CAE/CNN_converted.h5'))